<a href="https://colab.research.google.com/github/dhinesh2022h/chatbox1/blob/main/RAG_COPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.2 which is incompatible.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.28.2 which is incompatible.
tensorflow 2.17.0 requires protobuf!

In [ ]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = "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" #CHANGE API KEY, USE YOUR OWN


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [ ]:
!mkdir data
!gdown 18ODzbpu4bugyitbxqExsDyOCC2tS7BNj -O "data/rpp.pdf"

In [ ]:
import os

# Replace 'your_llama_parse_api_key_here' with your actual LLAMA_PARSE API key
os.environ["LLAMA_PARSE_API_KEY"] = "llx-lbjHUrKtdjTCfGU63zizgSlnY7innSdf9RfUPgfMpLtutE0L"   #CHANGE API KEY USE YOUR OWN !!!!+

In [ ]:
# instruction = """The provided document is NVIDIA's First Quarter Fiscal 2024 Financial Results.
# This is a press release that provides detailed financial information about NVIDIA's performance for the first quarter of its fiscal year 2024.
# It includes unaudited financial statements, management's commentary, highlights of key developments, and disclosures related to NVIDIA's outlook for the next quarter.
#  The document contains many financial tables and figures. Try to be precise while answering questions based on the information in this press release."""

instruction = """The provided document contains detailed financial information, including unaudited financial statements, management commentary, key highlights, and outlook disclosures related to a company's performance for a specific fiscal period. The document may contain various financial tables, figures, and data points. When answering questions based on the information in this document, strive to provide accurate and precise
responses by carefully referencing the relevant data and details presented in the document."""

parser = LlamaParse(
    api_key=os.getenv("LLAMA_PARSE_API_KEY"),
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("./data/rpp.pdf")

In [ ]:
parsed_doc = llama_parse_documents[0]

In [ ]:
Markdown(parsed_doc.text[:4096])

NVIDIA Announces Financial Results for First Quarter Fiscal 2024
Quarterly revenue of $7.19 billion, up 19% from previous quarter

Record Data Center revenue of $4.28 billion

Second quarter fiscal 2024 revenue outlook of $11.00 billion

NVIDIA (NASDAQ: NVDA) today reported revenue for the first quarter ended April 30, 2023, of $7.19 billion, down 13% from a year ago and up 19% from the previous quarter.

GAAP earnings per diluted share for the quarter were
0.82
,
u
p
28
0.82,up281.09, down 20% from a year ago and up 24% from the previous quarter.

“The computer industry is going through two simultaneous transitions — accelerated computing and generative AI,” said Jensen Huang, founder and CEO of NVIDIA.

“A trillion dollars of installed global data center infrastructure will transition from general purpose to accelerated computing as companies race to apply generative AI into every product, service and business process.

“Our entire data center family of products — H100, Grace CPU, Grace Hopper Superchip, NVLink, Quantum 400 InfiniBand and BlueField-3 DPU — is in production. We are significantly increasing our supply to meet surging demand for them,” he said.

During the first quarter of fiscal 2024, NVIDIA returned to shareholders $99 million in cash dividends.

NVIDIA will pay its next quarterly cash dividend of $0.04 per share on June 30, 2023, to all shareholders of record on June 8, 2023.

Q1 Fiscal 2024 Summary
|GAAP|(
i
n
m
i
l
l
i
o
n
s
,
e
x
c
e
p
t
e
a
r
n
i
n
g
s
p
e
r
s
h
a
r
e
)
∣
∣
∣
∣
∣
∣
−
−
−
∣
−
−
−
∣
−
−
−
∣
−
−
−
∣
−
−
−
∣
−
−
−
∣
∣
∣
Q
1
F
Y
24
∣
Q
4
F
Y
23
∣
Q
1
F
Y
23
∣
Q
/
Q
∣
Y
/
Y
∣
∣
R
e
v
e
n
u
e
∣
inmillions,exceptearningspershare)∣∣∣∣∣∣−−−∣−−−∣−−−∣−−−∣−−−∣−−−∣∣∣Q1FY24∣Q4FY23∣Q1FY23∣Q/Q∣Y/Y∣∣Revenue∣ 7,192|
6
,
051
∣
6,051∣ 8,288|Up 19%|Down 13%| |Gross margin|64.6%|63.3%|65.5%|Up 1.3 pts|Down 0.9 pts| |Operating expenses|
2
,
508
∣
2,508∣ 2,576|
3
,
563
∣
D
o
w
n
3
∣
O
p
e
r
a
t
i
n
g
i
n
c
o
m
e
∣
3,563∣Down3∣Operatingincome∣ 2,140|
1
,
257
∣
1,257∣ 1,868|Up 70%|Up 15%| |Net income|
2
,
043
∣
2,043∣ 1,414|
1
,
618
∣
U
p
44
∣
D
i
l
u
t
e
d
e
a
r
n
i
n
g
s
p
e
r
s
h
a
r
e
∣
1,618∣Up44∣Dilutedearningspershare∣ 0.82|
0.57
∣
0.57∣ 0.64|Up 44%|Up 28%|

|Non-GAAP|(
i
n
m
i
l
l
i
o
n
s
,
e
x
c
e
p
t
e
a
r
n
i
n
g
s
p
e
r
s
h
a
r
e
)
∣
(
inmillions,exceptearningspershare)∣( in millions, except earnings per share)|(
i
n
m
i
l
l
i
o
n
s
,
e
x
c
e
p
t
e
a
r
n
i
n
g
s
p
e
r
s
h
a
r
e
)
∣
(
inmillions,exceptearningspershare)∣( in millions, except earnings per share)| | | | | |---|---|---|---|---|---| |Q1 FY24|Q4 FY23|Q1 FY23|Q/Q|Y/Y| | |Revenue|
7
,
192
∣
7,192∣ 6,051|
8
,
288
∣
U
p
19
∣
G
r
o
s
s
m
a
r
g
i
n
∣
66.8
∣
O
p
e
r
a
t
i
n
g
e
x
p
e
n
s
e
s
∣
8,288∣Up19∣Grossmargin∣66.8∣Operatingexpenses∣ 1,750|
1
,
775
∣
1,775∣ 1,608|Down 1%|Up 9%| |Operating income|
3
,
052
∣
3,052∣ 2,224|
3
,
955
∣
U
p
37
∣
N
e
t
i
n
c
o
m
e
∣
3,955∣Up37∣Netincome∣ 2,713|
2
,
174
∣
2,174∣ 3,443|Up 25%|Down 21%| |Diluted earnings per share|
1.09
∣
1.09∣ 0.88|$ 1.36|Up 24%|Down 20%|

Outlook
NVIDIA’s outlook for the second quarter of fiscal 2024 is as follows:
Financial Overview
Revenue is expected to be
11.00
b
i
l
l
i
o
n
,
p
l
u
s
o
r
m
i
n
u
s
2
11.00billion,plusorminus22.71 billion and
1.90
b
i
l
l
i
o
n
,
r
e
s
p
e
c
t
i
v
e
l
y
.
G
A
A
P
a
n
d
n
o
n
−
G
A
A
P
o
t
h
e
r
i
n
c
o
m
e
a
n
d
e
x
p
e
n
s
e
a
r
e
e
x
p
e
c
t
e
d
t
o
b
e
a
n
i
n
c
o
m
e
o
f
a
p
p
r
o
x
i
m
a
t
e
l
y
1.90billion,respectively.GAAPandnon−GAAPotherincomeandexpenseareexpectedtobeanincomeofapproximately90 million, excluding gains and losses from non-affiliated investments. GAAP and non-GAAP tax rates are expected to be 14.0%, plus or minus 1%, excluding any discrete items.

Highlights
Data Center
First-quarter revenue was a record $4.28 billion, up 14% from a year ago and up 18% from the previous quarter. Launched four inference platforms that combine the company’s full-stack inference software with the latest NVIDIA Ada, NVIDIA Hopper™ and NVIDIA Grace Hopper™ processors. Announced that Google Cloud is the first cloud provider offering the new NVIDIA L4 Tensor Core GPU to accelerate generative AI applications. Introduced NVIDIA AI Foundations to help businesses create and operate custom large language models and generative AI models trained with their own proprietary data for domain-specific tasks. Unveiled the NVIDIA cuLitho software library for computational lithography to accelerate the design and manufacturing of next-gen semiconductors. Expanded its partners offering new products and s

In [ ]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

In [ ]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

In [ ]:
#Downloading embeddings
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [ ]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [ ]:
%%time
# query = "What is the most important innovation from Nvidia?"
query = "What is the most important innovation from Nvidia?"
similar_docs = qdrant.similarity_search_with_score(query)

In [ ]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

In [ ]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

In [ ]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

In [ ]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

In [ ]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [ ]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [ ]:
%%time
response = qa.invoke("What is the most significant innovation from Nvidia?")

In [ ]:
print_response(response)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [ ]:
%%time
response = qa.invoke("What is the revenue for 2024 and % change?")

In [ ]:
Markdown(response["result"])

In [ ]:
%%time
response = qa.invoke("What is the FY23?")

In [ ]:
print_response(response)

In [1]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)

NameError: name 'qa' is not defined

In [ ]:
print_response(response)

In [ ]:
%%time
response = qa.invoke("compare the Gross profit from 2022 and 2023?")

In [ ]:
Markdown(response["result"])

In [ ]:
%%time
response = qa.invoke("what is the share holding of board of directors Mr. P Arulsundaram")

In [ ]:
Markdown(response["result"])